# Tạo Iceberg Tables cho NER Analysis (Model Results)

Notebook này tạo các bảng Iceberg để lưu kết quả từ model NER theo schema trong hình.

## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Create_NER_Gold_Tables")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("Spark Session đã được khởi tạo với Nessie catalog!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

25/12/06 10:00:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Session đã được khởi tạo với Nessie catalog!
Spark Master: spark://spark-master:7077
Application ID: app-20251206100013-0005


## 2. Tạo Database/Namespace

In [2]:
# Tạo database gold_result_model_multi_task nếu chưa tồn tại
spark.sql("CREATE DATABASE IF NOT EXISTS nessie.gold_result_model_multi_task")
spark.sql("USE nessie.gold_result_model_multi_task")

print("Database 'gold_result_model_multi_task' đã được tạo và đang sử dụng!")

Database 'gold_result_model_multi_task' đã được tạo và đang sử dụng!


## 3. Tạo Tables theo Schema trong hình

### 3.1. Table Entity - Danh sách entities (Major, Organization, etc.)

In [3]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Entity (
    entityID INT,
    entityType STRING,
    entityName STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'Entity' đã được tạo!")

Table 'Entity' đã được tạo!


### 3.2. Table Topic - Danh sách topics

In [4]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Topic (
    topicID INT,
    topicName STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'Topic' đã được tạo!")

Table 'Topic' đã được tạo!


### 3.3. Table Post - Thông tin posts

In [5]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Post (
    postID STRING,
    description STRING,
    timePublish TIMESTAMP,
    likeCount INT,
    commentCount INT,
    shareCount INT,
    intent STRING,
    type STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
PARTITIONED BY (days(timePublish))
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'Post' đã được tạo!")

Table 'Post' đã được tạo!


### 3.4. Table Post_Entity - Quan hệ Post và Entity (M:N)

In [6]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Post_Entity (
    postID STRING,
    entityID INT,
    entityOrder INT,
    entityName STRING,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'Post_Entity' đã được tạo!")

Table 'Post_Entity' đã được tạo!


### 3.5. Table Post_Topic - Quan hệ Post và Topic (M:N)

In [7]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Post_Topic (
    postID STRING,
    topicID INT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'Post_Topic' đã được tạo!")

Table 'Post_Topic' đã được tạo!


## 4. Kiểm tra các Tables đã tạo

In [8]:
# Liệt kê tất cả tables trong database gold_result_model_multi_task
tables = spark.sql("SHOW TABLES IN nessie.gold_result_model_multi_task")
tables.show(truncate=False)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print("Đã tạo 5 tables:")
print("  1. Entity - Danh sách entities (Major, Organization, etc.)")
print("  2. Topic - Danh sách topics")
print("  3. Post - Thông tin posts")
print("  4. Post_Entity - Quan hệ Post ↔ Entity (M:N)")
print("  5. Post_Topic - Quan hệ Post ↔ Topic (M:N)")
print("="*60)

+----------------------------+-----------+-----------+
|namespace                   |tableName  |isTemporary|
+----------------------------+-----------+-----------+
|gold_result_model_multi_task|Entity     |false      |
|gold_result_model_multi_task|Post       |false      |
|gold_result_model_multi_task|Post_Entity|false      |
|gold_result_model_multi_task|Post_Topic |false      |
|gold_result_model_multi_task|Topic      |false      |
+----------------------------+-----------+-----------+


SUMMARY
Đã tạo 5 tables:
  1. Entity - Danh sách entities (Major, Organization, etc.)
  2. Topic - Danh sách topics
  3. Post - Thông tin posts
  4. Post_Entity - Quan hệ Post ↔ Entity (M:N)
  5. Post_Topic - Quan hệ Post ↔ Topic (M:N)


## 5. Xem Schema của từng Table

In [9]:
# Xem schema của từng table
tables_list = ["Entity", "Topic", "Post", "Post_Entity", "Post_Topic"]

for table_name in tables_list:
    print(f"\n{'='*80}")
    print(f"Schema of table: {table_name}")
    print('='*80)
    spark.sql(f"DESCRIBE nessie.gold_result_model_multi_task.{table_name}").show(truncate=False)


Schema of table: Entity
+----------+---------+-------+
|col_name  |data_type|comment|
+----------+---------+-------+
|entityID  |int      |NULL   |
|entityType|string   |NULL   |
|entityName|string   |NULL   |
|created_at|timestamp|NULL   |
|updated_at|timestamp|NULL   |
+----------+---------+-------+


Schema of table: Topic
+----------+---------+-------+
|col_name  |data_type|comment|
+----------+---------+-------+
|topicID   |int      |NULL   |
|topicName |string   |NULL   |
|created_at|timestamp|NULL   |
|updated_at|timestamp|NULL   |
+----------+---------+-------+


Schema of table: Post
+--------------+-----------------+-------+
|col_name      |data_type        |comment|
+--------------+-----------------+-------+
|postID        |string           |NULL   |
|description   |string           |NULL   |
|timePublish   |timestamp        |NULL   |
|likeCount     |int              |NULL   |
|commentCount  |int              |NULL   |
|shareCount    |int              |NULL   |
|intent     

## 6. Dừng Spark Session

In [10]:
# Dừng Spark Session để giải phóng resources
spark.stop()
print("Spark Session đã được dừng!")

Spark Session đã được dừng!
